    1_6_230104_013_seoul_man
     
     4-5 section
     
     data : /aiffel/socar_gis/data/korea_seoul_sig.json

In [1]:
#1. package load
import requests
import json
import os
import math
import time
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap
print('슝~')

슝~


In [2]:
#2. data load  ...SGIS data 가져오기 # SGIS API 사이트의 본인의 consumer_key, consumer_secret를 입력합니다.
consumer_key = '2f74504a63224ee09f2b'       # 서비스ID
consumer_secret =  '63a0ffef893e43a48ae2' # 보안Key


In [3]:
#2. data load  ... SGIS data 가져오기 # SGIS ACCESS TOKEN을 리턴 해주는 함수
def get_access_token(consumer_key, consumer_secret) -> None:
    temp_res = requests.get(
        url="https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json",
        params={
            "consumer_key": consumer_key,
            "consumer_secret": consumer_secret
        }
    )

    access_token = temp_res.json()["result"]["accessToken"]
   
    return access_token
print('슝~')

슝~


In [4]:
#7 서울시 구별 인구수 데이터 수집 및 시각화  .. 데이터 수집 및 전처리 .. 구단위 data
seoul_sig_geo_data_path = os.getenv('HOME') + '/aiffel/socar_gis/data/korea_seoul_sig.json'
seoul_sig_geo_data = json.load(
    open(seoul_sig_geo_data_path, encoding="utf-8")
)

print('슝~')

슝~


In [5]:
#7 서울시 구별 인구수 데이터 수집 및 시각화  ### SGIS ACCESS TOKEN을 리턴 해주는 함수
def get_access_token(consumer_key, consumer_secret) -> None:
    temp_res = requests.get(
        url="https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json",
        params={
            "consumer_key": consumer_key,
            "consumer_secret": consumer_secret
        }
    )

    access_token = temp_res.json()["result"]["accessToken"]
  
    return access_token

print('슝~')

슝~


In [6]:
#7 서울시 구별 인구수 데이터 수집 및 시각화 ### 서울시 구단위 인구주택총조사 정보 
# 응답정보 컬럼 설명 링크: https://sgis.kostat.go.kr/developer/html/openApi/api/data.html#4
def get_population_json(access_token, si_code) -> None:
    url="https://sgisapi.kostat.go.kr/OpenAPI3/stats/population.json"
    params={
        "accessToken": access_token,
        "year": 2019, # 데이터 기준연도 범위: 2000 ~ 2019 / 2019가 가장 최근 데이터
        "adm_cd": si_code, # 행정구역코드
        "low_search": 1 # 행정구역코드에 해당하는 정보만 요청: 0, 1단계 하위 행정구역 정보 요청: 1, 2단계 하위 행정구역 정보 요청: 2
    }

    temp_population_json = requests.get(
        url=url,
        params=params
    ).json()["result"]

    return temp_population_json

print('슝~')

슝~


In [7]:
#7 서울시 구별 인구수 데이터 수집 및 시각화   ... DataFrame을 생성 구코드(adm_cd), 구이름(adm_nm), 인구수(tot_ppltn). 컬럼 변경
# SGIS access_token 가져오기
access_token = get_access_token(consumer_key, consumer_secret)


# 서울시 구단위 행정구역 인구수 정보 데이터 프레임으로 읽어오기
seoul_gu_pop_df = pd.DataFrame(get_population_json(access_token, 11)) # 서울 시코드: 11


# 서울시 구코드(adm_cd), 구이름(adm_nm), 인구수(tot_ppltn) 컬럼만 남기기
seoul_gu_pop_df = seoul_gu_pop_df[["adm_cd", "adm_nm", "tot_ppltn"]]

# 컬럼명 변경
seoul_gu_pop_df.rename(
    columns={
        "adm_cd": "gu_cd",
        "adm_nm": "gu_nm"
    },
    inplace=True
)

seoul_gu_pop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   gu_cd      25 non-null     object
 1   gu_nm      25 non-null     object
 2   tot_ppltn  25 non-null     object
dtypes: object(3)
memory usage: 728.0+ bytes


In [8]:
#7 서울시 구별 인구수 데이터 수집 및 시각화  인구수(tot_ppltn) 컬럼은 데이터 타입을 int로 
seoul_gu_pop_df["tot_ppltn"] = seoul_gu_pop_df["tot_ppltn"].astype(int)
seoul_gu_pop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   gu_cd      25 non-null     object
 1   gu_nm      25 non-null     object
 2   tot_ppltn  25 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 728.0+ bytes


In [9]:
#7 서울시 구별 인구수 데이터 수집 및 시각화  .... map center 지정
# 서울시 중심부 위도, 경도
center = [37.541, 126.986]

# map이 서울 중심부에 위치하고, zoom 레벨은 10으로 시작 합니다.
map = folium.Map(
    location=center,
    zoom_start=10,
    )

print('슝~')

슝~


In [10]:
#7 서울시 구별 인구수 데이터 수집 및 시각화 .. gu data add
folium.Choropleth(
    geo_data=seoul_sig_geo_data,
    name="서울시 구단위 인구수",
    data=seoul_gu_pop_df,
    columns=["gu_nm", "tot_ppltn"],
    key_on="feature.id",
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    line_color='white', 
    line_weight=0,
    highlight=False, 
    smooth_factor=1.0,
    legend_name="인구수",
).add_to(map)



In [11]:
#7 서울시 구별 인구수 데이터 수집 및 시각화 . map
map